<a href="https://colab.research.google.com/github/ddeMoivre/Deep-learning/blob/master/Text_Classification_on_fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -q transformers

     |████████████████████████████████| 3.8 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 6.6 MB 35.5 MB/s 
     |████████████████████████████████| 596 kB 9.4 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 


In [2]:
#hide
! pip install -q datasets transformers

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 27.2 MB/s 
     |████████████████████████████████| 212 kB 47.3 MB/s 
     |████████████████████████████████| 136 kB 52.8 MB/s 
     |████████████████████████████████| 127 kB 46.4 MB/s 
     |████████████████████████████████| 94 kB 1.3 MB/s 
     |████████████████████████████████| 271 kB 42.5 MB/s 
     |████████████████████████████████| 144 kB 39.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
#hide 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#hide
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/NLP"

In [7]:
#hide
%cd /content/drive/MyDrive/NLP/

/content/drive/MyDrive/NLP


In [8]:
import transformers
import pandas as pd
from datasets import load_dataset, load_metric, ClassLabel

print(transformers.__version__)

4.17.0


In [9]:
from sklearn.utils import shuffle

def pre_process(filepath):
    dataset = []
    
    # Add labels
    true = pd.read_csv(os.path.join(filepath, 'True.csv'))
    true['label'] = 1
    fake = pd.read_csv(os.path.join(filepath, 'Fake.csv'))
    fake['label'] = 0
    
    # Merge two dataframes into one
    dataset = pd.concat([true, fake],join='outer',axis=0)
    dataset = shuffle(dataset, random_state=42)
    dataset = dataset.reset_index(drop=True)

    # Merge title and text
    dataset['text'] = dataset['title'] + '. ' + dataset['text']

    df = dataset.drop(['title','subject','date'], axis=1)
    #df.to_csv('news articles.csv', index=True)

    split_point = int(len(df)*.8)
    # Split
    train = df[:split_point]
    validation = df[split_point:]

    train.to_csv('train.csv', index=True, index_label='idx')
    validation.to_csv('validation.csv', index=True, index_label='idx')     
    
    return df

df = pre_process('/content/drive/MyDrive/NLP')


In [10]:
df.head()

,text,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,0
1,Failed GOP Candidates Remembered In Hilarious...,0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,0
3,California AG pledges to defend birth control ...,1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,0


In [11]:
data_files = {"train": "train.csv", "validation": "validation.csv"}
dataset = load_dataset('csv', data_files=data_files)

Using custom data configuration default-18d88fc616838ad4


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-18d88fc616838ad4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
dataset['train'].features

{'idx': Value(dtype='int64', id=None),
 'label': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None)}

In [13]:
new_features = dataset['train'].features.copy()
new_features['label'] = ClassLabel(num_classes=2, names=['fake', 'true'])
dataset['train'] = dataset['train'].cast(new_features)
dataset['train'].features

Casting the dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

{'idx': Value(dtype='int64', id=None),
 'label': ClassLabel(num_classes=2, names=['fake', 'true'], id=None),
 'text': Value(dtype='string', id=None)}

In [14]:
new_features = dataset['validation'].features.copy()
new_features["label"] = ClassLabel(num_classes=2, names=['fake', 'true'])
dataset['validation'] = dataset['validation'].cast(new_features)
dataset['validation'].features

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

{'idx': Value(dtype='int64', id=None),
 'label': ClassLabel(num_classes=2, names=['fake', 'true'], id=None),
 'text': Value(dtype='string', id=None)}

In [15]:
dataset['train'][0]

{'idx': 0,
 'label': 0,
 'text': ' BREAKING: GOP Chairman Grassley Has Had Enough, DEMANDS Trump Jr. Testimony. Donald Trump s White House is in chaos, and they are trying to cover it up. Their Russia problems are mounting by the hour, and they refuse to acknowledge that there are problems surrounding all of this. To them, it s  fake news,  or a  hoax.  However, the facts bear things out differently, and it seems that there are now cracks in the Congressional public leadership.Chuck Grassley (R-Iowa), who heads the Senate Judiciary Committee, is fed up. He is now demanding that Donald Trump, Jr. and former 2016 Trump Campaign Manager Paul Manafort testify before his committee regarding the now infamous shady meeting between Donald Trump and the shady Russian lawyer who promised dirt on 2016 Democratic Presidential nominee Hillary Clinton. In fact, this information is due, well, NOW. This demand sends a few signals to team Trump   most notably that they should not fire Special Counsel R

# Fine-tuning a model on a text classification task

In [17]:
task = "qqp"
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [18]:
#dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', task)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

To access an actual element, you need to select a split first, then give an index:

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [19]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [20]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.546875, 'f1': 0.5797101449275361}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [21]:
from transformers import AutoTokenizer
    
tokenizer = tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [22]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
task_to_keys = {
    "qqp": ("text"),
}

We can double check it does work on our current dataset:

In [24]:
sentence = task_to_keys[task]
print(f"Sentence: {dataset['train'][0][sentence]}")

Sentence:  BREAKING: GOP Chairman Grassley Has Had Enough, DEMANDS Trump Jr. Testimony. Donald Trump s White House is in chaos, and they are trying to cover it up. Their Russia problems are mounting by the hour, and they refuse to acknowledge that there are problems surrounding all of this. To them, it s  fake news,  or a  hoax.  However, the facts bear things out differently, and it seems that there are now cracks in the Congressional public leadership.Chuck Grassley (R-Iowa), who heads the Senate Judiciary Committee, is fed up. He is now demanding that Donald Trump, Jr. and former 2016 Trump Campaign Manager Paul Manafort testify before his committee regarding the now infamous shady meeting between Donald Trump and the shady Russian lawyer who promised dirt on 2016 Democratic Presidential nominee Hillary Clinton. In fact, this information is due, well, NOW. This demand sends a few signals to team Trump   most notably that they should not fire Special Counsel Robert Mueller under any 

We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [25]:
def preprocess_function(examples):
    
    return tokenizer(examples['text'], truncation=True, padding=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [26]:
preprocess_function(dataset['train'][1])

{'input_ids': [101, 3478, 2175, 2361, 5347, 4622, 1999, 26316, 19545, 7327, 21615, 1006, 2678, 1007, 1012, 2085, 2008, 6221, 8398, 2003, 1996, 3653, 17421, 24971, 2175, 2361, 9773, 1010, 2009, 1055, 2051, 2000, 3342, 2035, 2216, 2060, 5347, 2040, 2699, 2061, 2524, 2000, 3786, 2032, 1999, 1996, 2679, 2000, 1996, 2317, 2160, 1012, 2044, 2035, 1010, 2129, 2064, 2057, 5293, 2035, 1996, 3335, 2618, 4523, 1010, 11721, 16020, 2015, 1010, 6881, 2791, 1010, 1998, 11591, 8909, 3695, 9243, 1997, 2107, 5347, 2004, 15333, 2497, 5747, 1010, 8879, 14548, 3695, 1010, 2198, 10556, 19570, 2232, 1010, 6945, 8096, 1010, 3841, 9806, 1010, 1998, 18431, 10882, 28741, 2050, 1029, 2045, 1055, 1037, 2678, 2437, 1996, 6241, 2006, 10474, 2008, 2515, 2074, 2008, 1010, 1998, 7327, 21197, 10057, 2093, 1997, 2122, 3478, 5347, 2004, 2295, 2027, 2128, 2757, 1006, 1998, 1996, 2175, 2361, 2993, 2453, 2004, 2092, 2022, 2757, 2012, 2023, 2391, 4312, 1007, 1012, 23263, 4159, 1010, 1037, 7327, 6483, 2005, 1996, 2175, 2361, 1

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [27]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/36 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [28]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [29]:
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [30]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [31]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [32]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx. If text, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 35918
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11225


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.004500,0.001174,0.999666,0.999655
2,0.002800,0.000006,1.000000,1.000000
3,0.002000,0.000015,1.000000,1.000000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx. If text, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8980
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-qqp/checkpoint-2245
Configuration saved in bert-base-uncased-finetuned-qqp/checkpoint-2245/config.json
Model weights saved in bert-base-uncased-finetuned-qqp/checkpoint-2245/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-qqp/checkpoint-2245/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-qqp/checkpoint-2245/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx. If text, idx are not expected by `BertForSequenceClas

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.004500,0.001174,0.999666,0.999655
2,0.002800,0.000006,1.000000,1.000000
3,0.002000,0.000015,1.000000,1.000000
4,0.000000,0.000002,1.000000,1.000000
5,0.000000,0.000001,1.000000,1.000000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx. If text, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8980
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-qqp/checkpoint-8980
Configuration saved in bert-base-uncased-finetuned-qqp/checkpoint-8980/config.json
Model weights saved in bert-base-uncased-finetuned-qqp/checkpoint-8980/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-qqp/checkpoint-8980/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-qqp/checkpoint-8980/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx. If text, idx are not expected by `BertForSequenceClas

TrainOutput(global_step=11225, training_loss=0.002738963823844855, metrics={'train_runtime': 10789.3294, 'train_samples_per_second': 16.645, 'train_steps_per_second': 1.04, 'total_flos': 4.72521144321024e+16, 'train_loss': 0.002738963823844855, 'epoch': 5.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [33]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx. If text, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8980
  Batch size = 16


{'epoch': 5.0,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_loss': 6.253015726542799e-06,
 'eval_runtime': 173.1943,
 'eval_samples_per_second': 51.849,
 'eval_steps_per_second': 3.245}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("sgugger/my-awesome-model")
```

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

In [ ]:
# ! pip install optuna
# ! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10) 
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [ ]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()